In [1]:
# Install required packages
!pip install tensorflow==2.12.0
!pip install tensorflowjs==3.12.0
!pip install gdown

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os
import shutil

In [3]:
# Download dataset dari Google Drive
!gdown --id 1j3NCboyzjHCv2Lu1pZqF39ZfIKET7bMF

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1j3NCboyzjHCv2Lu1pZqF39ZfIKET7bMF
From (redirected): https://drive.google.com/uc?id=1j3NCboyzjHCv2Lu1pZqF39ZfIKET7bMF&confirm=t&uuid=6159abd5-ae40-4997-b3fd-8e5b2e4e13bf
To: /content/Dataset.zip
100% 223M/223M [00:02<00:00, 96.8MB/s]


In [4]:
# Ekstrak dataset ke dalam direktori /content
with zipfile.ZipFile('Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [5]:
# Path dataset
train_dir = '/content/Dataset/Train'
validation_dir = '/content/Dataset/Test'

In [6]:
# Inisialisasi ImageDataGenerator untuk augmentasi data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

# Persiapan data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [7]:
# Definisi model menggunakan Sequential API
model = Sequential([
    InputLayer(input_shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Kompilasi model dengan optimizer RMSprop, loss binary crossentropy
model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
# Melatih model dengan data generator
history = model.fit(
    train_generator,
    epochs=3,
    validation_data=validation_generator
)

Epoch 1/3
706/706 [==============================] - 833s 1s/step - loss: 0.4675 - accuracy: 0.7841 - val_loss: 0.3744 - val_accuracy: 0.8488
Epoch 2/3
706/706 [==============================] - 843s 1s/step - loss: 0.4208 - accuracy: 0.8132 - val_loss: 0.3800 - val_accuracy: 0.8532
Epoch 3/3
706/706 [==============================] - 841s 1s/step - loss: 0.4022 - accuracy: 0.8236 - val_loss: 0.3240 - val_accuracy: 0.8691


In [9]:
# Simpan model ke dalam format .h5 di direktori /content
model.save("/content/model.h5")

In [10]:
# Mengonversi model ke format TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Menyimpan model TFLite ke dalam file
with open('/content/model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model TFLite telah di-generate dan disimpan sebagai model.tflite")

Model TFLite telah di-generate dan disimpan sebagai model.tflite


In [11]:
# Mengonversi model ke format TensorFlow.js
!tensorflowjs_converter --input_format=keras /content/model.h5 /content/tfjs_model

# Download model TensorFlow.js
shutil.make_archive('/content/tfjs_model', 'zip', '/content/tfjs_model')

print("Model TensorFlow.js telah di-generate dan di-download sebagai tfjs_model.zip")

2024-06-15 05:16:58.560402: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Model TensorFlow.js telah di-generate dan di-download sebagai tfjs_model.zip
